## 对Jobs进行标记
### steps：
* 读入数据全部数据
* 基本的处理，时间作差，把numerical转化为catagorical
* 乱序
* 标记index，一共应该有4000行数据
* 遍历所有的数据，进行label
    * 如果index小于10，就输出前面所有的labor
    * 如果labor大于10，从[0,index-1]中随机取出10个显示在console上，并按照已经标记好的priority排序
    * 读取input，写入当前值进行标记

In [117]:
import numpy as np
import pandas as pd

--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user


In [94]:
data_all_df = pd.read_csv('./inputs/raw_data/all_table_3000_2.csv', index_col=0)
data_sla_df = pd.read_csv('./inputs/raw_data/sla_not_null_500_2.csv', index_col=0)
data_prixy_df = pd.read_csv('./inputs/raw_data/priorityxy_not_null_500_2.csv', index_col=0)
all_df = pd.concat((data_all_df,data_sla_df, data_prixy_df), axis=0)
all_df=all_df.reset_index(drop = True)

In [95]:
#判断是否存在SLA
all_df['isExceedSLA'] = 0
all_df['isExceedSLA']=pd.isnull(all_df['SLA']) 

In [96]:
all_df.head()

,JobId,JobStateId,Priority,TokenAllocation,QuotaTokens,AUId,VCId,TotalWaitingTimeInMins,random,SLA,JobRegistryId,IsYieldedEnabled,IsYieldForEnabled,YieldScopePriority_X,YieldScopePriority_Y,isExceedSLA
0,a6ea9d6e-27a8-4bc3-94c0-3fe7be9fbd01,0,2200,500,0,af2b0993-27ab-4d5b-bf57-114f767dc540,100,8206.650700,2d6b3993-bb07-4e3b-af83-00001092f7ba,NaN,4782,1,1,NaN,NaN,True
1,99adce30-dfb6-4dac-8022-4841af96b791,0,2200,500,0,af2b0993-27ab-4d5b-bf57-114f767dc540,100,293.254264,26dd8ccd-b8b0-4f28-92ab-00001172c903,NaN,4782,1,1,NaN,NaN,True
2,793c3838-6949-4a12-83f1-7ce4e882f649,0,2200,500,0,af2b0993-27ab-4d5b-bf57-114f767dc540,100,1674.709764,a375622b-d633-49f7-a7bf-00001672d5b0,NaN,4782,1,1,NaN,NaN,True
3,93d98cfc-eefd-4693-93d8-ea2d66973829,0,2053,500,0,af2b0993-27ab-4d5b-bf57-114f767dc540,100,0.478794,0088f297-aefc-4769-a9de-000016cb0681,NaN,4782,1,1,NaN,NaN,True
4,55f5249b-6b65-43d5-8d70-277fe1ad7797,0,2200,500,0,af2b0993-27ab-4d5b-bf57-114f767dc540,100,1637.284038,cf1a9d35-0970-4e42-9b81-0000201fe958,NaN,4782,1,1,NaN,NaN,True


In [97]:
all_df = all_df.drop(['JobId','Priority','AUId','VCId','random','JobRegistryId','SLA','JobStateId'],axis=1)
all_df.head()

,TokenAllocation,QuotaTokens,TotalWaitingTimeInMins,IsYieldedEnabled,IsYieldForEnabled,YieldScopePriority_X,YieldScopePriority_Y,isExceedSLA
0,500,0,8206.650700,1,1,NaN,NaN,True
1,500,0,293.254264,1,1,NaN,NaN,True
2,500,0,1674.709764,1,1,NaN,NaN,True
3,500,0,0.478794,1,1,NaN,NaN,True
4,500,0,1637.284038,1,1,NaN,NaN,True


In [98]:
all_df.describe()

,TokenAllocation,QuotaTokens,TotalWaitingTimeInMins,IsYieldedEnabled,IsYieldForEnabled,YieldScopePriority_X,YieldScopePriority_Y
count,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,947.000000,946.000000
mean,575.799750,7422.935500,686.212755,0.978500,0.991250,1.105597,0.090909
std,925.795006,6743.661208,2503.106494,0.145062,0.093143,0.352341,0.808802
min,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,234.000000,2339.000000,1.797872,1.000000,1.000000,1.000000,0.000000
50%,350.000000,7506.000000,13.299094,1.000000,1.000000,1.000000,0.000000
75%,500.000000,10164.000000,94.985452,1.000000,1.000000,1.000000,0.000000
max,8000.000000,29125.000000,34770.431422,1.000000,1.000000,2.000000,10.000000


In [99]:
# 添加一列AU是否满载，用0,1随机数来填充
all_df['IsQuotaFullyLoad']=np.random.randint(0,2,size=len(all_df))


In [100]:
all_df.describe()

,TokenAllocation,QuotaTokens,TotalWaitingTimeInMins,IsYieldedEnabled,IsYieldForEnabled,YieldScopePriority_X,YieldScopePriority_Y,IsQuotaFullyLoad
count,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,947.000000,946.000000,4000.000000
mean,575.799750,7422.935500,686.212755,0.978500,0.991250,1.105597,0.090909,0.503750
std,925.795006,6743.661208,2503.106494,0.145062,0.093143,0.352341,0.808802,0.500048
min,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,234.000000,2339.000000,1.797872,1.000000,1.000000,1.000000,0.000000,0.000000
50%,350.000000,7506.000000,13.299094,1.000000,1.000000,1.000000,0.000000,1.000000
75%,500.000000,10164.000000,94.985452,1.000000,1.000000,1.000000,0.000000,1.000000
max,8000.000000,29125.000000,34770.431422,1.000000,1.000000,2.000000,10.000000,1.000000


In [101]:
all_df.head()

,TokenAllocation,QuotaTokens,TotalWaitingTimeInMins,IsYieldedEnabled,IsYieldForEnabled,YieldScopePriority_X,YieldScopePriority_Y,isExceedSLA,IsQuotaFullyLoad
0,500,0,8206.650700,1,1,NaN,NaN,True,1
1,500,0,293.254264,1,1,NaN,NaN,True,1
2,500,0,1674.709764,1,1,NaN,NaN,True,1
3,500,0,0.478794,1,1,NaN,NaN,True,0
4,500,0,1637.284038,1,1,NaN,NaN,True,1


In [102]:
#乱序
all_df = all_df.sample(frac=1)

In [103]:
all_df.head()

,TokenAllocation,QuotaTokens,TotalWaitingTimeInMins,IsYieldedEnabled,IsYieldForEnabled,YieldScopePriority_X,YieldScopePriority_Y,isExceedSLA,IsQuotaFullyLoad
3755,1500,10164,14.566891,1,1,1.0,0.0,True,0
2848,350,2339,22.342987,1,1,NaN,NaN,True,0
2623,500,2339,0.789012,1,1,NaN,NaN,True,1
197,500,7635,13.872800,1,1,NaN,NaN,True,1
2459,234,2705,5.765547,1,1,NaN,NaN,True,1


In [104]:
all_df=all_df.reset_index(drop = True)

In [105]:
all_df['Priority'] = None
all_df.head()

,TokenAllocation,QuotaTokens,TotalWaitingTimeInMins,IsYieldedEnabled,IsYieldForEnabled,YieldScopePriority_X,YieldScopePriority_Y,isExceedSLA,IsQuotaFullyLoad,Priority
0,1500,10164,14.566891,1,1,1.0,0.0,True,0,None
1,350,2339,22.342987,1,1,NaN,NaN,True,0,None
2,500,2339,0.789012,1,1,NaN,NaN,True,1,None
3,500,7635,13.872800,1,1,NaN,NaN,True,1,None
4,234,2705,5.765547,1,1,NaN,NaN,True,1,None


In [114]:
[rows,columns]=[all_df.shape[0],all_df.shape[1]]
[rows,columns]

[4000, 10]

In [ ]:
# 遍历
# 前五十个用插入排序，剩下的根据随机显示10个进行排序
import pdb
pdb.set_trace()
import random
[rows,columns]=[all_df.shape[0],all_df.shape[1]]
for index in range(rows):
    if index<50:
        ##打印未排序的全部jobs
        control_group=all_df.iloc[0:index-1]
    else:
        ##仅打印排序的10个jobs
        collection=range(index)
        nums = random.sample(collection, 10)
        control_group=all_df.iloc[nums]
    # 这里应该加一个control 对 priority排序，但priority还没有添加这一
    control_group.sort_values(by="Priority" , ascending=True)
    print(control_group)
    str_input=str(index)+"priority："
    temp = input(str_input)
    all_df.iloc[index,'priority']=temp


--Return--
None
> <ipython-input-118-9e343832638e>(4)<module>()
      2 # 前五十个用插入排序，剩下的根据随机显示10个进行排序
      3 import pdb
----> 4 pdb.set_trace()
      5 import random
      6 [rows,columns]=[all_df.shape[0],all_df.shape[1]]



In [106]:
#保存dataframe
all_df.to_csv(path_or_buf='./labelled_data.csv')